In [1]:
import numpy as np
import pandas as pd 
import os
import random
import time
from tqdm import tqdm
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, SequentialSampler, RandomSampler, DataLoader
from torch.cuda.amp import autocast as autocast, GradScaler
from transformers import AutoConfig, AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, logging
from transformers import BertConfig, BertTokenizer, BertModel

from sklearn.metrics import precision_score,recall_score,f1_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

import gc 
gc.enable()



In [2]:

from datetime import datetime
from datetime import timedelta
from datetime import timezone
 
SHA_TZ = timezone(
    timedelta(hours=8),
    name='Asia/Shanghai',
)
utc_now = datetime.utcnow().replace(tzinfo=timezone.utc)

beijing_now = utc_now.astimezone(SHA_TZ)


# CONFIG

In [3]:

CONFIG=dict(
    DEBUG_MODEL=True,
    model_name="xlm-roberta-base",
    # model_name="bert-base-uncased",

    run_db=True,
    FP16=True,
    early_patience=10,
    LR=1e-5,
    EPS=1e-8,
    NUM_FOLDS=5,
    dropout_rate=0.0,
    TRAIN_BATCH_SIZE = 128,
    DEV_BATCH_SIZE = 1024,

    num_hidden_layers=12,
    multiple_language=True,
    cross_val=False,
    MAX_LENGTH = 32,
    SEED=42,
    EPOCHS = 100,
    DATA_NUM=150000,

    data_name="query",
    # data_name="imbd",
    INPUT_DIR="/vc_data/users/v-mxiong/qc/labeldata1",
    SCHEMA_DIR="/vc_data/users/v-mxiong/qc/schema1",
    MODEL_DIR="",
    SAVE_PATH="./checkpoint/datareview"
)

if not os.path.exists(CONFIG["SAVE_PATH"]):
    os.mkdir(CONFIG["SAVE_PATH"])


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.set_verbosity_error()

HASH_NAME=f"{CONFIG['model_name']}___{CONFIG['data_name']}:{beijing_now}"
CONFIG['group'] = f'{HASH_NAME}-Baseline'


    

In [4]:
import wandb

try:
    api_key = "ebe051612bfb733306f4e4b5df4b043050ebea6e"
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account,  \nGet your W&B access token from here: https://wandb.ai/authorize')


wandb: Currently logged in as: adreambear. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/v-mxiong/.netrc


In [5]:
if CONFIG["DEBUG_MODEL"]==True:
    CONFIG["TRAIN_BATCH_SIZE"] = 16
    CONFIG["DEV_BATCH_SIZE"] = 128
    CONFIG["NUM_FOLDS"]=5
    CONFIG["EPOCHS"]=100
    CONFIG["run_db"]=False


In [6]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['SEED'])

# readdata

In [7]:
def read_data_txt(input_file,schema_file,is_debug=CONFIG["DEBUG_MODEL"]):
    with open(schema_file,"r") as input_object:
        schema_line=input_object.readlines()
    head=schema_line[0].strip().split("\t")
    with open(input_file,"r") as input_object:
        input_lines=input_object.readlines()
    if CONFIG["DEBUG_MODEL"]==True:
        input_lines=input_lines[:CONFIG['DATA_NUM']]
    input_lines=[ line.strip().split("\t") for line in input_lines]
    input_lines=[ line for line in input_lines if len(line)==len(head)]
    data_rows=input_lines
    input_df=pd.DataFrame(data_rows,columns=head)
    
    return input_df
def sample_negative(df,negative_rate=1):
    negative_df=df[df.label==0]
    positive_df=df[df.label==1]
    sample_num=len(positive_df)*negative_rate
    
    sample_df=negative_df.sample(n=sample_num,random_state=CONFIG["SEED"])
    
    # .sample(frac=1) 用于打乱顺序
    return_df=pd.concat([positive_df,sample_df]).sample(frac=1)
    return_df=return_df.reset_index(drop=True)
    return return_df

import langid
from tqdm._tqdm_notebook import tqdm_notebook
# import fasttext
def language_classify(df):

    tqdm_notebook.pandas(desc='apply')
#     df["language"]=df.progress_apply(lambda r: langid.classify(r["Query"])[0], axis=1)
    df["language"]=df.Query.progress_apply(lambda r: langid.classify(r)[0])
    df_en=df[df["language"]=="en"]
        
    return df,df_en

def select_langugage(df,language="en"):

    tqdm_notebook.pandas(desc='apply')
    return_choose=input_df.Market.progress_apply(lambda r:r[0:2]==language)
    
    return_df=df[return_choose]
    print("filter rate:",len(return_df)/len(df))
    
    return return_df

# def language_fasttext_classify(df):
#     path_to_pretrained_model = '/vc_data/users/v-mxiong/model_file/lid.176.bin'
#     fmodel = fasttext.load_model(path_to_pretrained_model)
#     tqdm_notebook.pandas(desc='apply')
#     df["language"]=df.Query.progress_apply(lambda r: fmodel.predict(r)[0][0][9:])
#     df_en=df[df["language"]=="en"]
    
#     print("filter rate:",len(df_en)/len(df))
#     return df,df_en
# input_lang_df=language_classify(input_df)

def select_market(df,language="en"):

    tqdm_notebook.pandas(desc='apply')
    return_choose=input_df.Market.progress_apply(lambda r:r[0:2]==language)
    
    return_df=df[return_choose]
    print("filter rate:",len(return_df)/len(df))
    
    return return_df
# input_lang_df=language_classify(input_df)

In [8]:
if CONFIG["data_name"]=="query":
    input_df=read_data_txt(CONFIG["INPUT_DIR"],CONFIG["SCHEMA_DIR"])
#     _,input_df=language_classify(input_df)
    if CONFIG["multiple_language"]==False: 
        input_df=select_market(input_df)
    
    
    input_df["label"]=input_df["label"].replace(["0","1"],[0,1])
    input_df=sample_negative(input_df,1)
    print(input_df["label"].value_counts())

    data_df,test_df,y_train,y_test=train_test_split(input_df,input_df["label"],test_size=0.2,random_state=CONFIG["SEED"])
    data_df.reset_index(inplace=True,drop=True)
    test_df.reset_index(inplace=True,drop=True)
    print(len(data_df),len(test_df))
    CONFIG.update({"len(data_df)":len(data_df)})    

1    3284
0    3284
Name: label, dtype: int64
5254 1314


In [9]:
def get_data(datapath):
    pos_files = os.listdir(datapath + '/pos')
    neg_files = os.listdir(datapath + '/neg')
    print(len(pos_files))
    print(len(neg_files))

    pos_all = []
    neg_all = []
    for pf, nf in zip(pos_files, neg_files):
        with open(datapath + '/pos' + '/' + pf, encoding='utf-8') as f:
            s = f.read()
            pos_all.append(s)
        with open(datapath + '/neg' + '/' + nf, encoding='utf-8') as f:
            s = f.read()
            neg_all.append(s)

    X_orig= np.array(pos_all + neg_all)
    Y_orig = np.array([1 for _ in range(len(pos_all))] + [0 for _ in range(len(neg_all))])
    print("X_orig:", X_orig.shape)
    print("Y_orig:", Y_orig.shape)

    return X_orig, Y_orig

if CONFIG["data_name"]=="imbd":
    x_orig,y_orig=get_data('/vc_data/users/v-mxiong/imdb/aclImdb/train')
    x_test,y_test=get_data('/vc_data/users/v-mxiong/imdb/aclImdb/test')
    data_df=pd.DataFrame({"Query":x_orig,"label":y_orig})
    data_df=data_df.sample(frac=0.1).reset_index(drop=True)

    test_df=pd.DataFrame({"Query":x_test,"label":y_test})
    test_df=test_df.sample(frac=0.1).reset_index(drop=True)
    data_df["label"].value_counts()


In [10]:
kf = StratifiedKFold(n_splits=CONFIG["NUM_FOLDS"], shuffle=True, random_state=CONFIG["SEED"])

for f, (t_, v_) in enumerate(kf.split(data_df, data_df.label)):
    data_df.loc[v_, 'fold'] = f
data_df.fold.value_counts()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


3.0    1051
2.0    1051
1.0    1051
0.0    1051
4.0    1050
Name: fold, dtype: int64

# dataloader

In [11]:
class DatasetRetriever(Dataset):
    def __init__(self,data,tokenizer,max_len=CONFIG["MAX_LENGTH"],is_test=False):
        self.data=data
        self.tokenizer=tokenizer
        self.max_len=max_len
        self.text=data["Query"].values
        self.is_test=is_test
        if self.is_test==False:
            self.label=data["label"].values
    def __len__(self):
        return len(self.data)
    def __getitem__(self, item):
        text=self.text[item]

        features=self.convert_examples_to_features(text)
        features={"input_ids":features["input_ids"],"attention_mask":features["attention_mask"]}
        if self.is_test==False:
            label=self.label[item]
            return {"text":{key:torch.tensor(value,dtype=torch.long) for key,value in features.items()},
                    "label":torch.tensor(label,dtype=torch.long)}
        else:
            return {"text":{key:torch.tensor(value,dtype=torch.long) for key,value in features.items()}}
        
    def convert_examples_to_features(self, example):
        encoded = self.tokenizer.encode_plus(
            example,
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            is_split_into_words=False,
            )
        return encoded
def make_dataloader(data,batch_size,model_dir=CONFIG["model_name"],max_len=CONFIG["MAX_LENGTH"],is_test=False):
    tokenizer=AutoTokenizer.from_pretrained(model_dir)
    dataset=DatasetRetriever(data,tokenizer,max_len,is_test=is_test)
    sampler=SequentialSampler(dataset)
    
    dataloader=DataLoader(dataset,
                          batch_size=batch_size,
                          sampler=sampler
                         )
    return dataloader

In [12]:
def prepare_loaders(df,fold):
    df_train = df[df.fold != fold].reset_index(drop=True)
    df_valid = df[df.fold == fold].reset_index(drop=True)
    
    train_loader=make_dataloader(df_train,CONFIG["TRAIN_BATCH_SIZE"],CONFIG["model_name"],CONFIG["MAX_LENGTH"])
    
    valid_loader=make_dataloader(df_valid,CONFIG["DEV_BATCH_SIZE"],CONFIG["model_name"],CONFIG["MAX_LENGTH"])
    
    return train_loader, valid_loader

# Model

In [13]:
class OriginModel(nn.Module):
    def __init__(self,model_name=CONFIG["model_name"]):
        super(OriginModel,self).__init__()
        self.config=AutoConfig.from_pretrained(model_name)
        self.config.update({"num_hidden_layers":CONFIG["num_hidden_layers"]})
        self.model=AutoModel.from_pretrained(model_name,config=self.config)
        self.drop=nn.Dropout(p=CONFIG["dropout_rate"])
        
        self.linear=nn.Linear(self.config.to_dict()["hidden_size"],2)
           
        self.dense = nn.Linear(self.config.to_dict()["hidden_size"], self.config.to_dict()["hidden_size"])
        self.activation = nn.Tanh()
        self.loss_fn = torch.nn.CrossEntropyLoss()
        
    def forward(self,input_ids,attention_mask,labels=None):
        out=self.model(input_ids=input_ids,attention_mask=attention_mask,output_hidden_states=False)
        last_hidden_state = out[0]
        cls_embeddings = last_hidden_state[:,0]
#         pooled_output = self.dense(cls_embeddings)
#         pooled_output = self.activation(pooled_output)
        
#         out=self.drop(pooled_output)
        out=self.drop(cls_embeddings)
        
        outputs=self.linear(out)
        
        preds=outputs.squeeze(1)
        if labels!=None:
            loss=self.loss_fn(preds,labels)
            return loss
        else:
            return preds

In [14]:
def get_parameters(model,model_init_LR,multiplier, classifier_LR):
    #权重分层，越靠近下游学习率越高
    parameters=[]
    LR=model_init_LR
    # 迭代器包含 层名字和参数 parameters()函数只包含参数
    #定义的层字典，参数的key必须叫params，否则在optimizer 父类中冲突
    for layer in range(model.config.to_dict()["num_hidden_layers"]-1,-1,-1):
        layer_parameters={
            "params":[p for n,p in model.named_parameters() if f"encoder.layer.{layer}." in n],
            "LR":CONFIG["LR"]
        }
        LR*=multiplier
        parameters.append(layer_parameters)
    
    
    classify_parameters={
        #自己定义了什么分类层在此更改名字
        "params":[p for n,p in model.named_parameters() if "linear"  in n],
        "LR":classifier_LR
    }
        
    parameters.append(classify_parameters)
    return parameters

class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if np.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            print('terminating because of early stopping!')
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [15]:
def evaluate(model,dev_dataloader):
    model.eval()
    
    total_val_loss=0
    for batch in dev_dataloader:
        text_inputs=batch["text"]
        label=batch["label"].to(DEVICE)
        
        text_inputs={key: value.to(DEVICE) for key,value in text_inputs.items()}
        text_inputs["labels"]=label
        with torch.no_grad():
            loss=model(**text_inputs)
            
        total_val_loss+=loss.item()
        
    return total_val_loss/len(dev_dataloader)
def train(model,train_dataloader,dev_dataloader):
    
#     optimizer=AdamW(get_parameters(model, model_init_LR=CONFIG["LR"]*1.5, multiplier=0.975, classifier_LR=CONFIG["LR"]), LR = CONFIG["LR"], EPS= CONFIG["EPS"])
    optimizer=AdamW(model.parameters(), lr = CONFIG["LR"])
    """
    get_linear_schedule_with_warmup:学习率先从0开始warm_up到设定学习率，再逐渐减到0
    num_warmup_steps：完成预热的步数
    num_training_steps：训练批次*CONFIG["EPOCHS"] 训练的step数
    """
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, 
                                                num_training_steps=len(train_dataloader) * CONFIG["EPOCHS"])
    scaler = GradScaler(enabled=CONFIG["FP16"])
    
    best_val_loss=100
    best_acc_val=0
    best_model_dict=None
    
    es=EarlyStopping(patience=CONFIG["early_patience"])
    
    train_df,val_df=data_df[data_df.fold!=fold],data_df[data_df.fold==fold]

    if CONFIG["run_db"]==True:
        wandb.watch(model,log_freq=100)
    for epoch in range(CONFIG["EPOCHS"]):
        sum_train_loss=0
        start = time.time()
        model.train()
        
        for index,batch in enumerate(tqdm(train_dataloader,leave=False)):
            
#             model.zero_grad()
#             optimizer.zero_grad()
            text_inputs=batch["text"]
            label=batch["label"].to(DEVICE)
        
            text_inputs={key: value.to(DEVICE) for key,value in text_inputs.items()}
            text_inputs["labels"]=label
            with autocast(enabled=CONFIG["FP16"]):
                loss=model(**text_inputs)
            sum_train_loss+=loss.item()
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
#             loss.backward()
#             optimizer.step()
            optimizer.zero_grad()
            
            scheduler.step()
            
        avg_train_loss=sum_train_loss/len(train_dataloader)
        val_loss=evaluate(model,dev_dataloader)
        print(f"EPOCH:{epoch+1}/{CONFIG['EPOCHS']} train_loss:{avg_train_loss},val_loss:{val_loss}")
        
        _,_,acc_val=predict(val_df,None,model)
        print(f"EPOCH:{epoch+1}/{CONFIG['EPOCHS']} dev_acc:{acc_val}")
        
        time_used = time.time()-start
        print(f"one_epoch_time:{time_used}")
        
        if acc_val>best_acc_val:
            best_acc_val=acc_val

        if val_loss<best_val_loss:
            best_val_loss=val_loss
            best_model_dict=copy.deepcopy(model.state_dict())
            print(f"best_model saved ,val_loss:{val_loss}")

        if CONFIG["run_db"]==True:
            wandb.log({"Train LOSS":avg_train_loss,"Valid LOSS":val_loss})
            wandb.log({"Valid ACC":acc_val})
            run.summary["best_val_loss"]=best_val_loss
            run.summary["best_acc_val"]=best_acc_val

        if es.step(val_loss):
            break
    print(f"best_acc_val:{best_acc_val},best_val_loss:{best_val_loss}")
    return best_val_loss,best_model_dict

def predict(df,model_path=None,model=None):
    if model==None:
        print(f"\nUsing{model_path}")
        model=OriginModel(CONFIG["model_name"])
        model.load_state_dict(torch.load(model_path))
        model.to(DEVICE)
        
    model.eval()
    
    data_loader=make_dataloader(df,CONFIG["DEV_BATCH_SIZE"],CONFIG["model_name"],CONFIG["MAX_LENGTH"],is_test=True)
    predict_list=[]
    for batch in data_loader:
        text_inputs=batch["text"]
        text_inputs={key: value.to(DEVICE) for key,value in text_inputs.items()}

        with torch.no_grad():
            preds=model(**text_inputs)
            predict_list+=preds.cpu().detach().numpy().tolist()
    
    predictions=np.argmax(predict_list,axis=1)
    acc=(df["label"]==predictions).mean()

    
    gc.collect()
    
    return predict_list,predictions,acc
    


# Train

In [16]:
for fold in range(CONFIG["NUM_FOLDS"]):
    # 不用交叉验证
    if CONFIG["cross_val"]==False and fold>=1:
        break
    if CONFIG["run_db"]==True:
        run = wandb.init(project='Query_classify', 
                     config=CONFIG,
                     job_type='Train',
                     group=CONFIG['group'],
                     tags=[f"{CONFIG['model_name']}", f'{HASH_NAME}', 'cbe-loss'],
                     name=f'{HASH_NAME}-fold-{fold}',
                     anonymous='must')
    train_dataloader,dev_dataloader=prepare_loaders(data_df,fold)

    model=OriginModel(CONFIG["model_name"])
    model.to(DEVICE)
    print("training_start")
    val_loss,best_model_dict=train(model,train_dataloader,dev_dataloader)
    model_path=os.path.join(CONFIG["SAVE_PATH"],f"bestmodel{fold}.pth")
    torch.save(best_model_dict,model_path)
    
    _,_,acc_train_lastepoch=predict(data_df[data_df.fold!=fold],model_path,model)
    print(f"train{fold} acc in last epoch:{acc_train_lastepoch}")
    del model
    torch.cuda.empty_cache()
     
    gc.collect()
    if CONFIG["run_db"]==True:
        run.summary["lastepoch_train_acc"]=acc_train_lastepoch
        # wandb.log({"train acc in last epoch":acc_train_lastepoch})
        run.finish()

/opt/conda/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
  0%|          | 0/263 [00:00<?, ?it/s]

training_start


EPOCH:1/100 train_loss:0.7020800093280952,val_loss:0.6949464877446493


  1%|          | 2/263 [00:00<00:18, 14.21it/s]

EPOCH:1/100 dev_acc:0.5118934348239772
one_epoch_time:22.535662174224854
best_model saved ,val_loss:0.6949464877446493


...

EPOCH:13/100 dev_acc:0.5499524262607041
one_epoch_time:21.70734977722168


EPOCH:14/100 train_loss:0.272634334904726,val_loss:1.3565187851587932


  1%|          | 2/263 [00:00<00:17, 15.15it/s]

EPOCH:14/100 dev_acc:0.5547098001902949
one_epoch_time:21.9621479511261


EPOCH:15/100 train_loss:0.23468656100318572,val_loss:1.46173181798723
EPOCH:15/100 dev_acc:0.5404376784015223
one_epoch_time:22.183898448944092
terminating because of early stopping!
best_acc_val:0.5718363463368221,best_val_loss:0.6859789490699768
train0 acc in last epoch:0.9666904591958125


In [17]:
#no 交叉验证
if CONFIG["run_db"]==True:
    run = wandb.init(project='Query_classify', 
                 config=CONFIG,
                 job_type='dev',
                 group=CONFIG['group'],
                 tags=[f"{CONFIG['model_name']}", f'{HASH_NAME}', 'cbe-loss'],
                 name=f'{HASH_NAME}-fold-{fold}',
                 anonymous='must')

prediction_dev=np.zeros(len(data_df))
for fold in range(CONFIG["NUM_FOLDS"]):
    # 不用交叉验证
    if CONFIG["cross_val"]==False and fold>=1:
        break
    model_path=os.path.join(CONFIG["SAVE_PATH"],f"bestmodel{fold}.pth")
    
    train_df=data_df[data_df.fold!=fold]
    
    val_df = data_df[data_df.fold ==fold]
    val_index=val_df.index
    
    _,_,acc_train=predict(train_df,model_path,None)
    _,predictions,acc_val=predict(val_df,model_path)
    
    print(f"acc in train {fold}fold :{acc_train}")
    print(f"acc in val {fold}fold :{acc_val}")

    precesion=precision_score(val_df["label"], predictions, average='macro')
    recall=recall_score(val_df["label"], predictions, average='macro')
    f1=f1_score(val_df["label"], predictions, average='macro')
    
    print(f"precesion:{precesion},recall:{recall},f1:{f1}")
    prediction_dev[val_index]=predictions
    if CONFIG["run_db"]==True:
        run.summary[f"acc in train{fold}"]=acc_train
        run.summary[f"acc in val{fold}"]=acc_val
#         wandb.log({f"acc in train{fold}":acc_train,f"acc in val{fold}":acc_val})
        run.summary["precesion"]=precesion
        run.summary["recall"]=recall
        run.summary["f1"]=f1

if CONFIG["cross_val"]==False:
    precession=acc_val
else:
    precession=(data_df["label"]==prediction_dev).mean()
if CONFIG["run_db"]==True:
    run.summary["val_acc"]=precession
    # wandb.log({"acc in val":precession})
    run.finish()

torch.cuda.empty_cache()
gc.collect()


Using./checkpoint/datareview/bestmodel0.pth

Using./checkpoint/datareview/bestmodel0.pth
acc in train 0fold :0.6757078277420889
acc in val 0fold :0.570884871550904
precesion:0.571143665372253,recall:0.5708546080028969,f1:0.5704353184798518


0

# Test

In [18]:
# 测试集
predict_test=np.zeros([len(test_df),2])
for fold in range(CONFIG["NUM_FOLDS"]):
    if CONFIG["cross_val"]==False and fold>=1:
        break
    
    predict_list,predictions,acc_test=predict(test_df,model_path)
    predict_test+=np.asarray(predict_list)
    

prediction_test=np.argmax(predict_test,axis=1)
acc=(test_df["label"]==prediction_test).mean()

precesion=precision_score(test_df["label"], predictions, average='macro')
recall=recall_score(test_df["label"], predictions, average='macro')
f1=f1_score(test_df["label"], predictions, average='macro')

print(f"precesion in test data :{acc}")
print(f"precesion:{precesion},recall:{recall},f1:{f1}")
if CONFIG["run_db"]==True:
    run = wandb.init(project='Query_classify', 
                 config=CONFIG,
                 job_type='test',
                 group=CONFIG['group'],
                 tags=[f"{CONFIG['model_name']}", f'{HASH_NAME}', 'cbe-loss'],
                 name=f'{HASH_NAME}-fold-{fold}',
                 anonymous='must')
    run.summary["test_acc"]=acc
    run.summary["precesion"]=precesion
    run.summary["recall"]=recall
    run.summary["f1"]=f1

    # wandb.log({"acc":acc})
    run.finish()

torch.cuda.empty_cache()
gc.collect()


Using./checkpoint/datareview/bestmodel0.pth
precesion in test data :0.5669710806697108
precesion:0.5672429976023642,recall:0.5670527864332959,f1:0.5666977879544969


0